In [21]:
!pip install geopandas
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

In [102]:
import pandas as pd 


ais=pd.read_csv('/kaggle/input/ais-sample/2022_8100595_emissions_hourly.csv',low_memory=False).reset_index(drop=False) 
ports=pd.read_csv('/kaggle/input/best-centroids/best_centroids.csv')
ports.columns=['Unnamed: 0', 'PORT_CODE', 'LONGITUDE', 'LATITUDE']
#ports=pd.read_csv('/kaggle/input/tutti-i-porti/PORTS.csv')
ports.columns

Index(['Unnamed: 0', 'PORT_CODE', 'LONGITUDE', 'LATITUDE'], dtype='object')

In [81]:
ais

,index,IMO,MMSI,TIME,SOG,OBSERVED_SOG,LONGITUDE,LATITUDE,AIS_DRAUGHT,NAVIGATIONAL_STATUS,...,E_CO2_kg,E_SOx_kg,E_NOx_kg,E_PM_kg,E_PM10_kg,E_PM2.5_kg,E_CH4_kg,E_N2O_kg,E_BC_kg,PDemand_CorrFact
0,0,8100595,377901127,2022-01-01 00:00:00.000000,0.0,0,54.625,24.890625,5.800781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
1,1,8100595,377901127,2022-01-01 00:59:59.999999,0.0,0,54.625,24.890625,5.800781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
2,2,8100595,377901127,2022-01-01 02:00:00.000001,0.0,0,54.625,24.890625,5.800781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
3,3,8100595,377901127,2022-01-01 03:00:00.000000,0.0,0,54.625,24.890625,5.800781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
4,4,8100595,377901127,2022-01-01 03:59:59.999999,0.0,0,54.625,24.890625,5.800781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,8100595,377901127,2022-12-31 18:59:59.999999,0.0,0,72.375,21.734375,5.300781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
8756,8756,8100595,377901127,2022-12-31 20:00:00.000001,0.0,0,72.375,21.734375,5.300781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
8757,8757,8100595,377901127,2022-12-31 21:00:00.000000,0.0,0,72.375,21.734375,5.300781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0
8758,8758,8100595,377901127,2022-12-31 21:59:59.999999,0.0,0,72.375,21.734375,5.300781,dry_dock,...,741.132,2.326521,8.141,0.521774,0.521774,0.480032,0.0073,0.0372,0.05825,1.0


In [69]:
ports

,PORT_NAME,PORT_CODE,CNTR_CODE3,geometry,LONGITUDE,LATITUDE
0,PALAMOS,PAL,ESP,POINT (3.130501065132564 41.84464568199469),3.130501,41.844646
1,CHERBOURG,CER,FRA,POINT (-1.617801242477137 49.64976896138526),-1.617801,49.649769
2,DUBLIN,DUB,IRL,POINT (-6.21248660686723 53.34366540273503),-6.212487,53.343665
3,STRASBOURG,SXB,FRA,POINT (7.790219363505714 48.57710098988014),7.790219,48.577101
4,PIOMBINO,PIO,ITA,POINT (10.54559832492405 42.93688634781674),10.545598,42.936886
...,...,...,...,...,...,...
8124,HEKOU,HEK_2,CHN,POINT (118.9477555555185 38.07510308641481),118.947756,38.075103
8125,ZHENDONG,ZHE_5,CHN,POINT (120.2733143677931 34.30803971263794),120.273314,34.308040
8126,SVETLYY YAR,SVE_7,RUS,POINT (44.51325992421364 48.43336439390908),44.513260,48.433364
8127,TURKMENBASY,TUR_1,TKM,POINT (53.2216666667 39.35833333330002),53.221667,39.358333


Fai diventare

In [103]:
ais=ais[['index', 'LONGITUDE','LATITUDE','SOG']].copy()
ais['geometry'] = ais.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
ais_gdf = gpd.GeoDataFrame(ais, geometry='geometry')
ais_gdf.crs = "EPSG:4326"  # Coordinate system WGS 84, puoi cambiarlo a seconda delle tue esigenze



ports=ports[['PORT_CODE','LONGITUDE','LATITUDE']].copy()
ports['geometry'] = ports.apply(lambda row: Point(row['LONGITUDE'], row['LATITUDE']), axis=1)
ports_gdf = gpd.GeoDataFrame(ports, geometry='geometry')
ports_gdf.crs = "EPSG:4326"  # Coordinate system WGS 84, puoi cambiarlo a seconda delle tue esigenze

/opt/conda/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/conda/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


'ports=ports[[\'cluster\',\'CENTROID_LONGITUDE\',\'CENTROID_LATITUDE\']].copy()\nports[\'geometry\'] = ports.apply(lambda row: Point(row[\'CENTROID_LONGITUDE\'], row[\'CENTROID_LATITUDE\']), axis=1)\nports_gdf = gpd.GeoDataFrame(ports, geometry=\'geometry\')\nports_gdf.crs = "EPSG:4326"  # Coordinate system WGS 84, puoi cambiarlo a seconda delle tue esigenze\n'

# Faccio diventare i punti dei porti dei cerchi 

In [104]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import transform
import pyproj

# Convertire miglia nautiche in gradi (approssimazione)
def miles_to_degrees(miles):
    return miles / 60.0

radius_degrees = miles_to_degrees(10)  # raggio di 10 miglia nautiche

# Funzione per creare un cerchio attorno a un punto
def create_circle_around_point(point, radius):
    return point.buffer(radius)

# Applicare la funzione a tutti i punti dei porti
ports_gdf['geometry'] = ports_gdf['geometry'].apply(lambda point: create_circle_around_point(point, radius_degrees))

# Assicurati che il sistema di coordinate sia lo stesso per entrambe le GeoDataFrames
ports_gdf.crs = "EPSG:4326"
ports_gdf

,PORT_CODE,LONGITUDE,LATITUDE,geometry
0,0,-5.035156,51.68750,"POLYGON ((-4.86849 51.68750, -4.86929 51.67116..."
1,1,0.398193,53.62500,"POLYGON ((0.56486 53.62500, 0.56406 53.60866, ..."
2,2,-0.187744,53.62500,"POLYGON ((-0.02108 53.62500, -0.02188 53.60866..."
3,3,-3.029297,53.43750,"POLYGON ((-2.86263 53.43750, -2.86343 53.42116..."
4,4,-3.023438,53.43750,"POLYGON ((-2.85677 53.43750, -2.85757 53.42116..."
...,...,...,...,...
212,212,-5.882812,54.62500,"POLYGON ((-5.71615 54.62500, -5.71695 54.60866..."
213,213,-5.105469,55.59375,"POLYGON ((-4.93880 55.59375, -4.93960 55.57741..."
214,214,-1.463867,50.90625,"POLYGON ((-1.29720 50.90625, -1.29800 50.88991..."
215,215,1.336914,51.12500,"POLYGON ((1.50358 51.12500, 1.50278 51.10866, ..."


In [105]:
import geopandas as gpd
ais_gdf.crs = "EPSG:4326"
ports_gdf.crs = "EPSG:4326"
intersections = gpd.sjoin(ais_gdf, ports_gdf, how='left', predicate='intersects')
intersections_cleaned = intersections.drop_duplicates(subset=['index'])
print(intersections_cleaned)
intersections_cleaned.loc[intersections_cleaned['SOG'] > 3, 'PORT_CODE'] = 'in movimento '

      index  LONGITUDE_left  LATITUDE_left  SOG                   geometry  \
0         0          54.625      24.890625  0.0  POINT (54.62500 24.89062)   
1         1          54.625      24.890625  0.0  POINT (54.62500 24.89062)   
2         2          54.625      24.890625  0.0  POINT (54.62500 24.89062)   
3         3          54.625      24.890625  0.0  POINT (54.62500 24.89062)   
4         4          54.625      24.890625  0.0  POINT (54.62500 24.89062)   
...     ...             ...            ...  ...                        ...   
8755   8755          72.375      21.734375  0.0  POINT (72.37500 21.73438)   
8756   8756          72.375      21.734375  0.0  POINT (72.37500 21.73438)   
8757   8757          72.375      21.734375  0.0  POINT (72.37500 21.73438)   
8758   8758          72.375      21.734375  0.0  POINT (72.37500 21.73438)   
8759   8759          72.375      21.734375  0.0  POINT (72.37500 21.73438)   

      index_right  PORT_CODE  LONGITUDE_right  LATITUDE_right  

/tmp/ipykernel_36/2387593581.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'in movimento ' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  intersections_cleaned.loc[intersections_cleaned['SOG'] > 3, 'PORT_CODE'] = 'in movimento '


In [106]:
intersections_cleaned['PORT_CODE'].value_counts()
#np.sum(intersections_cleaned['PORT_CODE'].isna())

PORT_CODE
in movimento     1805
13.0               79
Name: count, dtype: int64

In [107]:
ports[ports['PORT_CODE']==13]

,PORT_CODE,LONGITUDE,LATITUDE,geometry
13,13,1.470703,51.40625,POINT (1.470703125 51.40625)


In [98]:
ports

,PORT_CODE,LONGITUDE,LATITUDE,geometry
0,PAL,3.130501,41.844646,POINT (3.130501065132564 41.84464568199469)
1,CER,-1.617801,49.649769,POINT (-1.617801242477137 49.64976896138526)
2,DUB,-6.212487,53.343665,POINT (-6.21248660686723 53.34366540273503)
3,SXB,7.790219,48.577101,POINT (7.790219363505714 48.57710098988014)
4,PIO,10.545598,42.936886,POINT (10.54559832492405 42.93688634781674)
...,...,...,...,...
8124,HEK_2,118.947756,38.075103,POINT (118.94775555551853 38.075103086414806)
8125,ZHE_5,120.273314,34.308040,POINT (120.27331436779312 34.308039712637935)
8126,SVE_7,44.513260,48.433364,POINT (44.51325992421364 48.43336439390908)
8127,TUR_1,53.221667,39.358333,POINT (53.2216666667 39.35833333330002)


In [99]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# Convertire il DataFrame in geometrie di punti
geometry = [Point(xy) for xy in zip(ports['LONGITUDE'], ports['LATITUDE'])]

# Creare un GeoDataFrame
gdf = gpd.GeoDataFrame(ports, geometry=geometry)

# Impostare un sistema di riferimento spaziale (SRID), ad esempio WGS84
gdf.crs = "EPSG:4326"

# Esportare come shapefile
gdf.to_file('output_shapefile.shp')
